In [1]:
!pip install transformers

In [2]:
import requests
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
import numpy as np
import matplotlib.pyplot as plt

In [3]:
labels = np.load("./data/data_y_2.npy", allow_pickle=True)

In [4]:
print(len(labels))

609


In [5]:
print(np.unique(labels))

['aluminum_can' 'boxes' 'glass_bottle' 'others' 'paper_cup'
 'plastic_bottle' 'soda_can']


In [6]:
labels[labels == 'soda_can'] = "aluminum soda can"
labels[labels == 'aluminum_can'] = "aluminum can"
labels[labels == 'boxes'] = "cardboard box"
labels[labels == 'glass_bottle'] = "glass bottle"
labels[labels == 'paper_cup'] = "paper cup"
labels[labels == 'plastic_bottle'] = "plastic bottle"
labels[labels == 'others'] = "other"

In [7]:
print(np.unique(labels))

['aluminum can' 'aluminum soda can' 'cardboard box' 'glass bottle' 'other'
 'paper cup' 'plastic bottle']


In [8]:
print(len(labels))

609


In [9]:
!huggingface-cli login --token YOUR_TOKEN --add-to-git-credential

Token is valid (permission: read).
The token `llama-3-2-11B-Vision-Instruct` has been saved to /home/ubuntu/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful.
The current active token is: `llama-3-2-11B-Vision-Instruct`


In [10]:
import torch
from transformers import pipeline

In [13]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
text=f"""
Recycling Guide:

- Paper: White and colored paper, Phone books, Magazines, Junk Mail, Envelopes, File folders, Cardboard (flattened), Cartons (milk, juice, & broth), \
Books (remove hard covers or binding)
- Cardboard boxes, e.g. cereal, etc.
- Plastic: Styrofoam packing blocks, Plastic medicine bottles (empty), Plastic bottles numbered 1-7 (remove plastic film or wrap) 
- Metal & Glass: Aluminum, steel, and tin cans, Clean aluminum foil, Glass bottles and jars (remove lids)
- Other: This item is not included in the this of acceptable recyclables, please dispose of it in the garbage.
"""
print(text)


Recycling Guide:

- Paper: White and colored paper, Phone books, Magazines, Junk Mail, Envelopes, File folders, Cardboard (flattened), Cartons (milk, juice, & broth), Books (remove hard covers or binding)
- Cardboard boxes, e.g. cereal, etc.
- Plastic: Styrofoam packing blocks, Plastic medicine bottles (empty), Plastic bottles numbered 1-7 (remove plastic film or wrap) 
- Metal & Glass: Aluminum, steel, and tin cans, Clean aluminum foil, Glass bottles and jars (remove lids)
- Other: This item is not included in the this of acceptable recyclables, please dispose of it in the garbage.



In [15]:
prompt= f"""
Your task is to determine if the item in the question can be recycled using the text from the Recycling Guide given below. Output your response as a single word, either "Yes" or "No".

{text}

"""
print(prompt)


Your task is to determine if the item in the question can be recycled using the text from the Recycling Guide given below. Output your response as a single word, either "Yes" or "No".


Recycling Guide:

- Paper: White and colored paper, Phone books, Magazines, Junk Mail, Envelopes, File folders, Cardboard (flattened), Cartons (milk, juice, & broth), Books (remove hard covers or binding)
- Cardboard boxes, e.g. cereal, etc.
- Plastic: Styrofoam packing blocks, Plastic medicine bottles (empty), Plastic bottles numbered 1-7 (remove plastic film or wrap) 
- Metal & Glass: Aluminum, steel, and tin cans, Clean aluminum foil, Glass bottles and jars (remove lids)
- Other: This item is not included in the this of acceptable recyclables, please dispose of it in the garbage.





In [16]:
messages = []
for i in range(len(labels)):
    q = f"Question: Can a {labels[i][0]} be recycled?"
    prompt_msg = prompt + q
    messages.append({ "role": "user", "content": prompt_msg })

print(messages[467])

{'role': 'user', 'content': '\nYour task is to determine if the item in the question can be recycled using the text from the Recycling Guide given below. Output your response as a single word, either "Yes" or "No".\n\n\nRecycling Guide:\n\n- Paper: White and colored paper, Phone books, Magazines, Junk Mail, Envelopes, File folders, Cardboard (flattened), Cartons (milk, juice, & broth), Books (remove hard covers or binding)\n- Cardboard boxes, e.g. cereal, etc.\n- Plastic: Styrofoam packing blocks, Plastic medicine bottles (empty), Plastic bottles numbered 1-7 (remove plastic film or wrap) \n- Metal & Glass: Aluminum, steel, and tin cans, Clean aluminum foil, Glass bottles and jars (remove lids)\n- Other: This item is not included in the this of acceptable recyclables, please dispose of it in the garbage.\n\n\nQuestion: Can a glass bottle be recycled?'}


In [17]:
print(len(messages))

609


In [18]:
from string import punctuation

pred_labels = []

for i in range(len(messages)):
    step = i % 10
    if step == 0:
        print("processed", i)
    step = i % 50
    if step == 0 and i > 0:
        # open file
        with open('bin_class_palm_desert_results_other_simple_nodelimit.txt', 'a') as f: 
            # write elements of list
            for item in range(i-50, i):
                f.write('%s\n' %pred_labels[item])
            print("File written successfully")
        # close the file
        f.close()
    outputs = pipe(
        [ messages[i] ],
        max_new_tokens=20,
    )

    pred = outputs[0]["generated_text"][-1]['content']
    pred = pred.strip(punctuation)
    pred_labels.append(pred)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


processed 0


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


processed 10


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


processed 590


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


processed 600
File written successfully


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [19]:
print(len(pred_labels))

609


In [20]:
true_labels = []
for i in range(len(labels)):
    if labels[i][0] == 'paper cup' or labels[i][0] == 'other':
        true_labels.append('No')
    else:
        true_labels.append('Yes')

print(len(true_labels))

609


In [21]:
print(true_labels[245:262])

['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']


In [22]:
print(np.unique(true_labels))

['No' 'Yes']


In [23]:
unique_values, counts = np.unique(true_labels, return_counts=True)
print(unique_values)
print(counts)

['No' 'Yes']
[139 470]


In [24]:
unique_values, counts = np.unique(pred_labels, return_counts=True)
print(unique_values)
print(counts)

['No' 'Yes']
[139 470]


In [25]:
from sklearn.metrics import classification_report

print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

          No       1.00      1.00      1.00       139
         Yes       1.00      1.00      1.00       470

    accuracy                           1.00       609
   macro avg       1.00      1.00      1.00       609
weighted avg       1.00      1.00      1.00       609



In [26]:
# open file
with open('bin_class_palm_desert_results_other_simple_nodelimit.txt', 'w+') as f:
    
    # write elements of list
    for item in pred_labels:
        f.write('%s\n' %item)
    
    print("File written successfully")


# close the file
f.close()

File written successfully


In [27]:
cook_county_guide=f"""
Recycling Guide:

- Metal: Metal objects that are able to be recycled includes steel and aluminum containers and foil.
- Paper: Paper that can be recycled includes cardboard (flattened), office paper, newspaper, and magazines.
- Glass: Glass items that can be recycled includes glass bottles and jars only
- Plastic: Plastic items that can be recycled includes plastic bottles, tubs, jugs, and jars only. Always replace your cap onto the empty and clean plastic bottle it came from. Then, place into your recycling bin to ensure both plastic components are successfully processed at your local recycling facility.
- Coffee cup: Your cup of joe is a recycling no. Lids, straws, and hot beverage cups should be kept out of your curbside recycling bin. \
  Despite the paper composition of hot beverage cups, they contain a lining that degrades the overall quality of the recycling batch and should not be recycled in your bin. \
  For iced coffee, some uncertainty remains about whether the plastic cup is recyclable. \
  Please check with your local authority to determine whether plastic coffee cups can go in your bin. \
  To reduce waste and save resources, use a reusable coffee cup and straw! Some coffee establishments even offer discounts when you bring your own cup/mug.
- Other: This item is not included in the this of acceptable recyclables, please dispose of it in the garbage.
"""

In [28]:
cook_county_prompt= prompt= f"""
Your task is to determine if the item in the question can be recycled using the text from the Recycling Guide given below. Output your response as a single word, either "Yes" or "No".

{cook_county_guide}

"""
print(cook_county_prompt)


Your task is to determine if the item in the question can be recycled using the text from the Recycling Guide given below. Output your response as a single word, either "Yes" or "No".


Recycling Guide:

- Metal: Metal objects that are able to be recycled includes steel and aluminum containers and foil.
- Paper: Paper that can be recycled includes cardboard (flattened), office paper, newspaper, and magazines.
- Glass: Glass items that can be recycled includes glass bottles and jars only
- Plastic: Plastic items that can be recycled includes plastic bottles, tubs, jugs, and jars only. Always replace your cap onto the empty and clean plastic bottle it came from. Then, place into your recycling bin to ensure both plastic components are successfully processed at your local recycling facility.
- Coffee cup: Your cup of joe is a recycling no. Lids, straws, and hot beverage cups should be kept out of your curbside recycling bin.   Despite the paper composition of hot beverage cups, they cont

In [29]:
messages = []
for i in range(len(labels)):
    q = f"Question: Can a {labels[i][0]} be recycled?"
    prompt_msg = cook_county_prompt + q
    messages.append({ "role": "user", "content": prompt_msg })

print(messages[467])

{'role': 'user', 'content': '\nYour task is to determine if the item in the question can be recycled using the text from the Recycling Guide given below. Output your response as a single word, either "Yes" or "No".\n\n\nRecycling Guide:\n\n- Metal: Metal objects that are able to be recycled includes steel and aluminum containers and foil.\n- Paper: Paper that can be recycled includes cardboard (flattened), office paper, newspaper, and magazines.\n- Glass: Glass items that can be recycled includes glass bottles and jars only\n- Plastic: Plastic items that can be recycled includes plastic bottles, tubs, jugs, and jars only. Always replace your cap onto the empty and clean plastic bottle it came from. Then, place into your recycling bin to ensure both plastic components are successfully processed at your local recycling facility.\n- Coffee cup: Your cup of joe is a recycling no. Lids, straws, and hot beverage cups should be kept out of your curbside recycling bin.   Despite the paper comp

In [30]:
print(len(messages))

609


In [31]:
from string import punctuation

pred_labels = []

for i in range(len(messages)):
    step = i % 10
    if step == 0:
        print("processed", i)
    step = i % 50
    if step == 0 and i > 0:
        # open file
        with open('bin_class_cook_county_results_other_simple_nodelimit.txt', 'a') as f: 
            # write elements of list
            for item in range(i-50, i):
                f.write('%s\n' %pred_labels[item])
            print("File written successfully")
        # close the file
        f.close()
    outputs = pipe(
        [ messages[i] ],
        max_new_tokens=20,
    )

    pred = outputs[0]["generated_text"][-1]['content']
    pred = pred.strip(punctuation)
    pred_labels.append(pred)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


processed 0


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


processed 10


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


processed 590


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


processed 600
File written successfully


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [32]:
print(len(pred_labels))

609


In [33]:
unique_values, counts = np.unique(pred_labels, return_counts=True)
print(unique_values)
print(counts)

['No' 'Yes']
[139 470]


In [34]:
from sklearn.metrics import classification_report

print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

          No       1.00      1.00      1.00       139
         Yes       1.00      1.00      1.00       470

    accuracy                           1.00       609
   macro avg       1.00      1.00      1.00       609
weighted avg       1.00      1.00      1.00       609



In [35]:
# open file
with open('bin_class_cook_county_results_other_simple_nodelimit.txt', 'w+') as f:
    
    # write elements of list
    for item in pred_labels:
        f.write('%s\n' %item)
    
    print("File written successfully")


# close the file
f.close()

File written successfully


In [36]:
print(np.unique(labels, return_counts=True))

(array(['aluminum can', 'aluminum soda can', 'cardboard box',
       'glass bottle', 'other', 'paper cup', 'plastic bottle'],
      dtype=object), array([ 70, 100, 100, 100,  80,  59, 100]))


In [37]:
metrics = {}
names, counts = np.unique(labels, return_counts=True)
for x,y in zip(names, counts):
    metrics[x] = { 'count': y.item(), 'correct': 0 }

In [38]:
print(metrics)

{'aluminum can': {'count': 70, 'correct': 0}, 'aluminum soda can': {'count': 100, 'correct': 0}, 'cardboard box': {'count': 100, 'correct': 0}, 'glass bottle': {'count': 100, 'correct': 0}, 'other': {'count': 80, 'correct': 0}, 'paper cup': {'count': 59, 'correct': 0}, 'plastic bottle': {'count': 100, 'correct': 0}}


In [39]:
for i in range(len(pred_labels)):
    if pred_labels[i] == true_labels[i]:
        metrics[labels[i][0]]['correct'] +=1

In [40]:
print(metrics)

{'aluminum can': {'count': 70, 'correct': 70}, 'aluminum soda can': {'count': 100, 'correct': 100}, 'cardboard box': {'count': 100, 'correct': 100}, 'glass bottle': {'count': 100, 'correct': 100}, 'other': {'count': 80, 'correct': 80}, 'paper cup': {'count': 59, 'correct': 59}, 'plastic bottle': {'count': 100, 'correct': 100}}


In [41]:
for x in metrics:
    print("accuracy: {0}: {1}".format(x, (metrics[x]['correct'] / metrics[x]['count']) * 100))

accuracy: aluminum can: 100.0
accuracy: aluminum soda can: 100.0
accuracy: cardboard box: 100.0
accuracy: glass bottle: 100.0
accuracy: other: 100.0
accuracy: paper cup: 100.0
accuracy: plastic bottle: 100.0


In [ ]:
from numpy import loadtxt

preds = loadtxt('bin_class_cook_county_results_other_simple_2.txt', dtype=str)
print(preds.shape)
print(preds[:10])